In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [31]:
# Đọc dữ liệu
vn_index = pd.read_csv(r'VN-Index.csv',index_col=0, parse_dates=True)
vic = pd.read_csv(r'VIC.csv',index_col=0, parse_dates=True)
vnb10y = pd.read_csv(r'VNB10Y.csv',index_col=0, parse_dates=True)

In [32]:
vn_index, vic, vnb10y = vn_index['Lần cuối'],vic['Lần cuối'],vnb10y['Lần cuối']

In [33]:
capm_model = pd.concat([vn_index,vic,vnb10y],axis=1)
capm_model.columns = ['VN_Index', 'VIC', 'VN_Bond']
capm_model.index.set_names('Date',inplace=True)

In [34]:
capm_model.dropna(axis=0,inplace=True)

In [37]:
capm_model

,VN_Index,VIC,VN_Bond
Date,,,
2013-01-08,492.43,33946.0,8.869
2013-01-10,492.24,33144.0,8.862
2013-01-11,497.08,35817.0,8.878
2013-02-08,494.66,33946.0,8.968
2013-02-10,494.39,33411.0,8.903
...,...,...,...
2021-12-11,1473.37,94500.0,2.030
2021-12-13,1476.21,103200.0,2.001
2021-12-14,1476.02,102500.0,2.001


In [36]:
def fix_format_number(df, columns):
    list_fixed = []
    for i in df[columns]:
        i = i.replace(',','')
        list_fixed.append(float(i))

    return list_fixed

capm_model["VIC"] = fix_format_number(capm_model,"VIC")
capm_model['VN_Index']= fix_format_number(capm_model,"VN_Index")

In [25]:
capm_model = capm_model.pct_change().dropna()
capm_model = capm_model.resample('W-Fri').ffill()

In [27]:
capm_model['Risk premium for VIC'] = capm_model['VIC'] - capm_model['VN_Bond'] 
capm_model['Market risk premium'] = capm_model['VN_Index'] - capm_model['VN_Bond'] 

In [28]:
capm_model

,VN_Index,VIC,VN_Bond,Risk premium for VIC,Market risk premium
Date,,,,,
2013-01-11,0.009833,0.080648,0.001805,0.078843,0.008027
2013-01-18,0.009833,0.080648,0.001805,0.078843,0.008027
2013-01-25,0.009833,0.080648,0.001805,0.078843,0.008027
2013-02-01,0.009833,0.080648,0.001805,0.078843,0.008027
2013-02-08,-0.004868,-0.052238,0.010137,-0.062375,-0.015006
...,...,...,...,...,...
2021-11-19,-0.011893,0.003175,-0.007976,0.011151,-0.003916
2021-11-26,-0.005184,0.047872,-0.002480,0.050352,-0.002704
2021-12-03,-0.008991,-0.074305,-0.010487,-0.063818,0.001495


In [29]:
r_vic = capm_model['Risk premium for VIC']
r_vnb10y  = capm_model['Market risk premium']

r_vnb10y, r_vic = np.array(r_vnb10y), np.array(r_vic)
r_vnb10y = sm.add_constant(r_vnb10y)
# Mô hình OLS
results = sm.OLS(r_vic,r_vnb10y).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.506
Model:                            OLS   Adj. R-squared:                  0.505
Method:                 Least Squares   F-statistic:                     475.6
Date:                Tue, 18 Jan 2022   Prob (F-statistic):           3.85e-73
Time:                        15:06:57   Log-Likelihood:                 638.69
No. Observations:                 467   AIC:                            -1273.
Df Residuals:                     465   BIC:                            -1265.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0026      0.003      0.897      0.3